# Notebook auxiliar

In [1]:
import json
import torch
import torch.utils.data as data
import torchaudio
from matplotlib import pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


#### La función `collate_fn` (para construir minibatches con datos de diferente longitud)

In [2]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader

class VariableLengthDataset(Dataset):
    def __init__(self): # Inventamos un dataset de longitud variable
        self.data = [
            torch.randint(0, 10, (length,)) for length in [5, 10, 8, 6, 12]
        ]
        self.labels = torch.randint(0, 2, (len(self.data),))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

def collate_fn(batch):
    # El batch es una lista de tuplas: [(dato1,label1), (dato2,label2),...]
    sequences, labels = zip(*batch) # Esto devuelve: 
                                    # sequences = (dato1,dato2,...)
                                    # labels = (label1,label2,...)
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0)
    labels = torch.tensor(labels)
    return padded_sequences, labels # Esta es la salida del dataloader

dataset = VariableLengthDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

for i in range(len(dataset)):
    dato,label =  dataset[i]
    print(dato,label)
print()

for batch_idx, (padded_sequences, labels) in enumerate(dataloader):
    print(f"Batch {batch_idx + 1}:")
    print("Padded Sequences:")
    print(padded_sequences)
    print("Labels:")
    print(labels)
    print("Shape of padded sequences:")
    print(padded_sequences.shape)

tensor([2, 0, 9, 8, 8]) tensor(1)
tensor([5, 1, 3, 3, 8, 2, 3, 1, 0, 0]) tensor(1)
tensor([6, 0, 5, 8, 8, 6, 2, 8]) tensor(1)
tensor([5, 6, 1, 3, 8, 6]) tensor(1)
tensor([7, 8, 3, 3, 4, 9, 6, 7, 2, 8, 7, 1]) tensor(0)

Batch 1:
Padded Sequences:
tensor([[2, 0, 9, 8, 8, 0, 0, 0, 0, 0],
        [5, 1, 3, 3, 8, 2, 3, 1, 0, 0]])
Labels:
tensor([1, 1])
Shape of padded sequences:
torch.Size([2, 10])
Batch 2:
Padded Sequences:
tensor([[6, 0, 5, 8, 8, 6, 2, 8],
        [5, 6, 1, 3, 8, 6, 0, 0]])
Labels:
tensor([1, 1])
Shape of padded sequences:
torch.Size([2, 8])
Batch 3:
Padded Sequences:
tensor([[7, 8, 3, 3, 4, 9, 6, 7, 2, 8, 7, 1]])
Labels:
tensor([0])
Shape of padded sequences:
torch.Size([1, 12])


In [3]:
for i in range(len(dataset)):
    dato,label =  dataset[i]
    print(dato,label)

tensor([2, 0, 9, 8, 8]) tensor(1)
tensor([5, 1, 3, 3, 8, 2, 3, 1, 0, 0]) tensor(1)
tensor([6, 0, 5, 8, 8, 6, 2, 8]) tensor(1)
tensor([5, 6, 1, 3, 8, 6]) tensor(1)
tensor([7, 8, 3, 3, 4, 9, 6, 7, 2, 8, 7, 1]) tensor(0)


##### Sobre `zip` y el operador `*`

In [4]:
# Si pasamos una lista como argumento a una función y le aplicamos el operador * a dicha lista,
# el operador "desarma" la lista y la convierte en argumentos separados para la función
x = ['a','b', 'c']
print(*x)
print(x[0],x[1],x[2])

# zip es un iterador de tuplas:
# tuplas de entrada: [(a,1),(b,2),(c,3)]
# tuplas de salida: ([a,b,c),(1,2,3)]
l = [('x',1),('y',2),('z',3)]
ll = zip(*l)
for i in ll:
    print(i)

a b c
a b c
('x', 'y', 'z')
(1, 2, 3)


##### Sobre tteradores e iterables
  - Un iterable es cualquier objeto que se pueda recorrer o *iterar*, por ejemplo una lista, una tupla, un rango, un diccionario. 
  - Un iterador es un objeto que implementa el protocolo de *iterador*, es decir, un objeto que tiene definidos los métodos:
    - `__iter__` que devuelve el propio objeto iterador
    - `__next__` que devuelve el siguiente elemento de la secuencia y un `StopIteration` cuando la secuencia llegó a su fin
  - La estructura de control `for`:
    - recibe un iterable, por ejemplo `range(10)`
    - le aplica `iter()` con lo cual lo convierte en iterador
    - llama repetidamente a `next()` hasta que aparezca la excepción `StopIteration` y termina
     

In [1]:
L = [1,2,3,4,5] # L es un iterable
iter_L = iter(L) # iter_L es un iterador
next(iter_L) # Devuelve el primer elemento del iterador

1

##### Sobre los generadores
Un generador es un objeto de tipo iterador creado por:
  - *Una función generadora*. Es decir, una función que devuelve sus resultados usando `yield` en lugar de `return`. Esto hace que la función recuerde el estado cuando se la deja y lo retome cuando se la llame de nuevo. 
  - *Una expresión generadora*. Es igual que la comprehension list solo que se usan paréntesis en lugar de corchetes.

Ojo, no todo objeto de tipo iterador es un objeto generador, solo los que son creados por una función generadora.

También se podría crear un generador como instancia de una clase siempre que dicha tenga implementado los métodos `__iter__` y `__next__` para convertirla en iterador y los elementos del objeto sean creados con un método generador, es decir una función de que devuelva mediante `yield`.


In [ ]:
# Función generadora
def even_numbers(limit):
    num = 0
    while num < limit:
        yield num
        num += 2
gen_even = even_numbers(10) # gen_even es un generador
for n in even_numbers(10):
    print(n)

print(next(gen_even),next(gen_even)) # Devuelve el siguiente elemento del generador


0
2
4
6
8
0 2


In [ ]:
# Expresión generadora
gen_even = (num for num in range(10) if num % 2 == 0) 
for n in gen_even:
    print(n)


0
2
4
6
8


In [8]:
# Clase que se comporta como un iterador
class NumerosCuadrados:
    def __init__(self, limite):
        self.limite = limite
        self.actual = 0 # Maneja el estado

    def __iter__(self):
        return self # Un iterador es un iterable que se devuelve a sí mismo

    def __next__(self):
        if self.actual < self.limite:
            valor = self.actual * self.actual
            self.actual += 1 # Actualiza el estado para la próxima llamada
            return valor
        else:
            raise StopIteration

# Uso de la clase iterador
cuadrados_obj = NumerosCuadrados(5)

for num in cuadrados_obj:
    print(num)

print(type(cuadrados_obj)) # Salida: <class '__main__.NumerosCuadrados'>

0
1
4
9
16
<class '__main__.NumerosCuadrados'>


In [ ]:
# Clase que contiene un método generador
# Para iterar con una instancia de esta 
# clase no es necesario implementar los métodos __iter__ y __next__
# porque el método generador ya lo hace implícitamente ya que todo generador es un iterador.
class SecuenciaPersonalizada:
    def __init__(self, inicio, fin):
        self.inicio = inicio
        self.fin = fin

    def generar_rango(self): # Esto es un método generador
        actual = self.inicio
        while actual <= self.fin:
            yield actual
            actual += 1

# Uso de la clase con un método generador
mi_secuencia = SecuenciaPersonalizada(1, 5)

# Llamar al método generador para obtener un objeto generador
gen_obj = mi_secuencia.generar_rango()

print(type(gen_obj)) # Salida: <class 'generator'>

for num in gen_obj:
    print(num)

<class 'generator'>
1
2
3
4
5


##### Diferencia entre generadores y clases/instancias
  - Clases/Instancias: Mantienen el estado de los datos (variables de instancia) a través de múltiples llamadas a sus métodos. Los métodos, a menos que usen yield, se ejecutan de forma completa cada vez.
  - Generadores: Mantienen el estado de la ejecución (variables locales, punto de pausa) y permiten "pausar" y "reanudar" la ejecución de la función.

Puedes tener métodos de clase que sean generadores (si contienen yield), pero esto es una elección de diseño específica y no una propiedad inherente de todos los métodos de clase.

Recordar que `__iter__` siempre debe devolver un objeto de tipo iterador. En el siguiente ejemplo esto lo hacemos devolviendo con `yield` ya que de este modo se convierte a `__iter__` en una función generadora y por lo tanto devolverá un iterador (de tipo generador)

In [13]:
a = [1,2,3,4,5]
class Prueba():
    def __init__(self, x):
        self.x = a

    def __iter__(self):
    #    return self.x
        for i in self.x:
            yield  -i
p = Prueba(a)

for i in p:
    print(i)  # Salida: 1, 2, 3, 4, 5

-1
-2
-3
-4
-5


##### La función `stack` (unión de tensores)

In [ ]:
import torch 
  
# creating tensors 
x = torch.tensor([1.,3.,6.,10.]) 
y = torch.tensor([2.,7.,9.,13.]) 
# printing above created tensors 
print("Tensor x:", x) 
print("Tensor y:", y) 
  
# join above tensor using "torch.stack()" 
print("join tensors:") 
t = torch.stack((x,y)) 
  
# print final tensor after join 
print(t) 


print("join tensors dimension 0:") 
t = torch.stack((x,y), dim = 0) 
print(t) 
  
print("join tensors dimension 1:") 
t = torch.stack((x,y), dim = 1) 
print(t) 

#### Reproducibilidad de los experimentos
Como fijar la semilla y aplicarla a todas las posibles variaciones aleatorias

In [ ]:
import torch
import numpy as np
import random # También para operaciones random de Python

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed) # Para todas las GPUs
    torch.cuda.manual_seed_all(seed) # Para múltiples GPUs
    np.random.seed(seed)
    random.seed(seed)
    # Algunas operaciones de cuDNN pueden ser no deterministas, se recomienda esto:
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # Puede hacer que el entrenamiento sea más lento

# Definir la semilla que usaremos
MY_SEED = 42
set_seed(MY_SEED)

# Acá empieza el código de la aplicación

In [2]:
import torch
import torchaudio
import torchaudio.transforms as T
import torchaudio.functional as F
import torch.nn as nn

import jiwer 
# Pruebitas para definir 'cer'
reference = 'chau'
cer_score = F.edit_distance('ciao', reference)/len(reference)
print(cer_score)
jcer_score = jiwer.cer(reference, 'ciao')
print(jcer_score)

# Pero en realidad en este programa uso el wer, porque el GreedyDecoder nos da las salidas en caracteres separados por blancos
# (como si fueran palabras). Y jiwer tiene los dos, así que usamos el wer del jiwer. 
# Después sirve como excusa para ver los alineamientos. 
def cer(pred,ref):
    return(jiwer.wer(ref, pred))
    # return(F.edit_distance(pred, ref)/len(ref))

0.5
0.5


##### Sobre `model.train()`, `model.eval()` y `with torch.no_grad()`
  - Usar `model.train()` cuando entrenamos para que `BatchNormalization` y `dropout` funcionen correctamente
  - Usar `model.eval()` cuando hacemos test o validación.
  - Usar `with torch.no_grad()` es decir no calcular el gradiente dentro de lo que esté en el bloque `with`

In [ ]:
# Training mode
model.train()
# Your training loop
# ...
# Now switch to evaluation mode for validation
model.eval()

with torch.no_grad():  # No gradient calculation for evaluation
    out_data = model(data)

# Don't forget to switch back to training mode!
model.train()


##### Esquema general
  - De alguna manera generamos el dataset que consta de (tensores):
      - (x_train, y_train)
      - (x_valid, y_valid)
      - (x_test, y_test)
  - Lo convertimos en un dataset que es un wrap que permite iterar sobre los datos. Podemos hacerlo nosotros o podemos usar `TensorDataset`
  - Convertimos el dataset en un dataloader que es una versión del dataset separada en batches. Esto lo hacemos con `DataLoader`. Si es necesario, a `DataLoader` les podemos pasar una función `collate_fn` que por ejemplo haga un padding si los datos no son todos de la misma longitud. También puede hacer un shuffle de los datos en cada epoch, lo cual es bueno en el entrenamiento. El bs de validación se puede hacer más grande ya que no necesita calcular gradientes ni hacer back propagation.

  Las siguientes sentencias:
  
    - `loss_fn = nn.CrossEntropyLoss()`
    - `loss_func = torch.nn.functional.cross_entropy`

  Hacen lo mismo, pero loss_fn es un objeto y loss_func es una función. Es un tema de programación estructurada, las dos hacen lo mismo

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch import optim

train_ds = TensorDataset(x_train, y_train)
valid_ds = TensorDataset(x_valid, y_valid)
test_ds = TensorDataset(x_test, y_test)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=bs*2)
test_dl = DataLoader(test_ds, batch_size=bs)

def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)
    opt = optim.SGD(model.parameters(), lr=lr) 

    if opt is not None:
        loss.backward()
        opt.step() #for p in model.parameters(): p -= p.grad * lr
        opt.zero_grad()

    return loss.item(), len(xb)

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    opt = optim.SGD(model.parameters(), lr=lr) 
    test_loss, correct = 0, 0
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl: # Batch de train
            pred = model(xb)
            loss = loss_func(pred, yb)
            loss.backward()
            opt.step() #for p in model.parameters(): p -= p.grad * lr
            opt.zero_grad()
        model.eval()
        with torch.no_grad():
            for xb, yb in valid_dl:
                pred = model(xb)
                test_loss += loss_func(pred,yb).item()

            correct += (pred.argmax(1) == yb).type(torch.float).sum().item()

            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)

tensor(1.)

In [ ]:
import torch
a = torch.tensor([0.1,0.7,0.3,0,0.9])
a.argmax().type(float32) == 3

tensor(False)

In [25]:
au = {'key': 'mbwm0_si1934', 'scored': True, 'hyp_absent': False, 'hyp_empty': False, 'num_edits': 7, 
 'num_ref_tokens': 13, 'WER': 53.84615384615385, 'insertions': 0, 'deletions': 7, 'substitutions': 0, 
 'alignment': [(...), (...), (...), (...), (...), (...), (...), (...), (...), (...), (...), (...), (...)], 
 'ref_tokens': [['sil', 'w', 'ey', 'dx', 'ah', 'l', 'ih', 'dx', 'l', 'w', 'ay', 'l', 'sil']], 
 'hyp_tokens': [['sil', 'w', 'ey', 'w', 'l', 'sil']]}
summary = {'WER': 55.38594854019464, 'SER': 100.0, 'num_edits': 8309, 'num_scored_tokens': 15002, 
           'num_erroneous_sents': 400, 'num_scored_sents': 400, 'num_absent_sents': 0, 
           'num_ref_sents': 400, 'insertions': 191, 'deletions': 5415, 
           'substitutions': 2703, 'error_rate': 55.38594854019464}


In [26]:
import speechbrain
cer_stats = speechbrain.utils.metric_stats.ErrorRateStats()
cer_stats.append(ids=['au'], predict = au['hyp_tokens'], target = au['ref_tokens'])
cer_stats.summarize()

{'WER': 53.84615384615385,
 'SER': 100.0,
 'num_edits': 7,
 'num_scored_tokens': 13,
 'num_erroneous_sents': 1,
 'num_scored_sents': 1,
 'num_absent_sents': 0,
 'num_ref_sents': 1,
 'insertions': 0,
 'deletions': 7,
 'substitutions': 0,
 'error_rate': 53.84615384615385}

In [1]:
# Creación  de un archivo JSON

try:
    with open('p.json', 'x') as f:
        f.write("""
{
"mrws1_sx320": {
"wav": "/dbase/timit/test/dr5/mrws1/sx320.wav",
"duration": 3.28325,
"spk_id": "mrws1",
"phn": "sil dh ih n ih r ih s ih n ih sil g aa sil m ey n aa sil b iy w ih th ih n w aa sil k ih ng sil d ih s sil t ih n sil s sil",
"wrd": "the nearest synagogue may not be within walking distance",
"ground_truth_phn_ends": "2360 2840 3216 4511 5556 7018 7880 10440 11160 12040 13160 13960 14200 17640 18280 19160 20360 21560 23800 25320 25720 26520 27800 28825 30440 31248 32208 34130 35880 36760 37640 37960 39101 40120 40360 41640 43000 43320 44200 44440 45280 46680 49560 52480"
},
} """)
except FileExistsError:
    print("p.json already exists. Skipping dummy file creation.")
        


p.json already exists. Skipping dummy file creation.


## Creación de un greedy ctc decoder

In [ ]:
import torch

def greedy_ctc_decode(emissions: torch.Tensor, blank_idx: int) -> list[str]:
    """
    Performs greedy CTC decoding on a batch of log-probabilities.

    Args:
        emissions: Tensor of shape (seq_len, batch_size, num_classes)
                   containing log-probabilities.
        blank_idx: Index of the blank token.

    Returns:
        A list of decoded strings, one for each sequence in the batch.
    """
    decoded_sequences = []
    # Permute to (batch_size, seq_len, num_classes) for easier argmax
    emissions = emissions.permute(1, 0, 2)

    for i in range(emissions.shape[0]): # Iterate over batch
        # Get the index of the max probability at each timestep
        argmax_preds = emissions[i].argmax(dim=-1)

        decoded_seq = []
        last_char_idx = -1
        for char_idx in argmax_preds:
            if char_idx != blank_idx and (char_idx != last_char_idx or last_char_idx == blank_idx):
                # Add if not blank and not a repeated character (unless the last was blank)
                decoded_seq.append(char_idx.item())
            last_char_idx = char_idx

        # Convert indices to actual characters (you'll need your vocab mapping)
        # For this example, let's assume `tokens` list is available
        decoded_strings = [tokens[idx] for idx in decoded_seq]
        decoded_sequences.append("".join(decoded_strings))
    return decoded_sequences

# Example usage
tokens = ["<blank>", "a", "b", "c", " "] # Your actual vocabulary
blank_idx = tokens.index("<blank>")

# Example emissions (seq_len, batch_size, num_classes)
# Let's say the model outputs: "a-a-b-blank-b-c" (where '-' is blank)
# This should decode to "abc"
emissions_example = torch.zeros(7, 1, len(tokens))
emissions_example[0, 0, tokens.index("a")] = 10
emissions_example[1, 0, blank_idx] = 10
emissions_example[2, 0, tokens.index("a")] = 10
emissions_example[3, 0, blank_idx] = 10
emissions_example[4, 0, tokens.index("b")] = 10
emissions_example[5, 0, blank_idx] = 10
emissions_example[6, 0, tokens.index("c")] = 10

# Add a small amount of noise to other classes to avoid all zeros in softmax
emissions_example += torch.randn_like(emissions_example) * 0.1

decoded_greedy = greedy_ctc_decode(emissions_example, blank_idx)
print(f"Greedy decoded: {decoded_greedy}")